# Video

> Class to process frames from a video file or camera.

In [ ]:
#| default_exp video

In [ ]:
#| export
import cv2
import numpy as np
import threading
import logging
import torch
from PIL import ImageChops, Image
from typing import Optional, Union, List, Dict

In [ ]:
#| export
def interpolate_images(image1, image2, alpha):
    """
    Interpolates two images with a given alpha blending factor.
    Args:
        image1: Input image which can be a NumPy array, PyTorch tensor, or PIL image.
        image2: Input image which can be a NumPy array, PyTorch tensor, or PIL image.
        alpha: Alpha blending factor, a float between 0 and 1.

    Returns:
        A PIL Image object representing the blended image.
    """
    # Convert PyTorch tensors to PIL images
    if torch.is_tensor(image1):
        # Ensure tensor is on CPU and in byte format
        if image1.device != torch.device('cpu'):
            image1 = image1.to('cpu')
        image1 = image1.numpy() if image1.requires_grad else image1.detach().numpy()
        image1 = Image.fromarray(np.uint8(image1))

    if torch.is_tensor(image2):
        # Ensure tensor is on CPU and in byte format
        if image2.device != torch.device('cpu'):
            image2 = image2.to('cpu')
        image2 = image2.numpy() if image2.requires_grad else image2.detach().numpy()
        image2 = Image.fromarray(np.uint8(image2))

    # Convert NumPy arrays to PIL images
    if isinstance(image1, np.ndarray):
        image1 = Image.fromarray(image1)

    if isinstance(image2, np.ndarray):
        image2 = Image.fromarray(image2)

    image1 = image1.resize(image2.size)

    # Perform the blending
    return ImageChops.blend(image1, image2, alpha)



def convert_to_pil_image(image):
    """
    Converts an input image which can be a NumPy array, PyTorch tensor, or PIL image to a PIL Image.
    """
    if isinstance(image, Image.Image):
        return image
    elif torch.is_tensor(image):
        # Convert PyTorch tensor to PIL Image
        if image.device != torch.device('cpu'):
            image = image.to('cpu')
        image = image.numpy() if image.requires_grad else image.detach().numpy()
        if image.ndim == 3 and image.shape[0] in {1, 3}:
            # Handle channels of tensor assuming CHW format
            image = image.transpose(1, 2, 0)  # Convert to HWC
        if image.shape[2] == 1:  # If single-channel image, convert to grayscale
            image = image[:, :, 0]
        return Image.fromarray((image * 255).astype(np.uint8) if image.dtype == np.float32 else image)
    elif isinstance(image, np.ndarray):
        # Convert NumPy array to PIL Image
        if image.ndim == 3 and image.shape[2] == 1:  # If single-channel image
            image = image[:, :, 0]
        return Image.fromarray(image)
    else:
        raise TypeError("Unsupported image type")



In [ ]:
#| export
logger = logging.getLogger(__name__)

class VideoStreamer:
    "Continuously reads frames from a video capture source."
    
    class Break(Exception):
        "Custom exception to break out of the video capture loop."
        pass
    
    def __init__(
            self,
            video_source: Union[str, int],
            frame_buffer_size: int = 10,
            capture_props: Optional[Dict[str, Union[int, float]]] = None,
            event = None,
        ):
        "Initializes a video stream with a frame buffer and optional capture properties."
        
        # camera and buffer settings
        self.video_source = video_source
        self.capture_props = capture_props or {}
        self.cap = None

        # buffer settings
        self.frame_buffer_size = frame_buffer_size
        self.current_frame = None
        self.frame_buffer = []

        # handles thread in the background
        self.thread = None
        self.stop_event = event or threading.Event()
    
    def start(self):
        "Start capturing video frames in a background thread."
        self.cap = cv2.VideoCapture(self.video_source)
        if not self.cap.isOpened():
            logger.error(f"Failed to open video source: {self.video_source}")
            raise ValueError(f"Failed to open video source: {self.video_source}")
        # setup and start camera
        self._set_capture_props()
        self._start_frame_thread()
    
    def _set_capture_props(self):
        "Set the video capture properties based on the provided configuration."
        for prop, value in self.capture_props.items():
            self.cap.set(getattr(cv2, prop), value)
        # view some info about the opened camera
        fps = int(self.cap.get(cv2.CAP_PROP_FPS))
        logger.info(f"Video source opened with FPS: {fps}")
    
    def _start_frame_thread(self):
        "Start the background thread for reading frames."
        self.thread = threading.Thread(target=self._read_frames, daemon=True)
        self.thread.start()
        logger.info(f"Started video capture from source: {self.video_source}")
    
    def _read_frames(self):
        "Reads frames until signalled to stop."
        while not self.stop_event.is_set():
            ret, frame = self.cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                self.current_frame = frame
                self._update_frame_buffer(frame)
            else:
                logger.warning(f"Failed to read frame from video source: {self.video_source}")
                break
            # small check for graceful shutdown
            if self.stop_event.is_set():
                break
    
    def _update_frame_buffer(self, frame):
        "Updates the frame buffer with the latest frame."
        if len(self.frame_buffer) >= self.frame_buffer_size:
            self.frame_buffer.pop(0)
        self.frame_buffer.append(frame)
    
    def get_current_frame(self) -> Optional[np.ndarray]:
        "Get the current frame from the video capture."
        return self.current_frame
    
    def get_frame_buffer(self) -> List[np.ndarray]:
        "Get the frame buffer containing the most recent frames."
        return self.frame_buffer
    
    def stop(self):
        "Stop the video capture and release resources."
        self.stop_event.set()
        self._join_frame_thread()
        self._release_capture()
        logger.info(f"Stopped video capture from source: {self.video_source}")
    
    def _join_frame_thread(self):
        "Wait for the frame reading thread to finish."
        if self.thread and self.thread.is_alive():
            self.thread.join()
    
    def _release_capture(self):
        "Release the video capture resources."
        if self.cap:
            self.cap.release()
    
    def __enter__(self):
        "Context manager entry point."
        self.start()
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        "Context manager exit point."
        self.stop()
    
    def __del__(self):
        "Destructor to ensure resources are released."
        self.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()